## Coursera - IBM Data Science Capstone Project ##

### Brian Lee ###
BrianTrewLee@gmail.com



LinkedIn: https://www.linkedin.com/in/brian-lee-64a2022b/

## 1. Introduction ##

Where in the world can you find a StarBuck's? Well... everywhere... almost. Is there a science to locating a StarBuck's? Can we create a model to predict where a StarBuck's is likely to be located?

In [ ]:
# import libraries

import urllib.request # open and read URLs

import json # handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # handle requests
import pandas as pd # process data as dataframes with Pandas
import numpy as np # handle data in a vectorized manner with NumPy

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't installed the GeoPy geocoding library yet
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't installed the Folium library yet
import folium # map rendering library

# Matplolib plotting library and associated modules
import matplotlib.pyplot as plt 
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # for K-Means clustering with Scikit-Learn

print("Libraries imported!")

Solving environment: | 

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
# Define a function to gather recommended venues, with specifically the name and category, using the explore API

def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=300, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=750, limit=100):
#def getNearbyVenues(latitudes, longitudes, radius=2000, limit=100):
    
    count = 0
    venues_list = []
    for lat, lng in zip(latitudes, longitudes):
        print(lat,'-',lng)
         
        #create a unique namefor this pointthat is easier todeal with than a combination of lat and lon
        point_name = "point"+str(count)
            
        # create the API request URL
        url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        #print("resultscols=",results.columns)
        

        
        # return only relevant information for each nearby venue
        venues_list.append([(
            point_name,
            lat, 
            lng, 
            v["venue"]["name"], 
            v["venue"]["location"]["lat"], 
            v["venue"]["location"]["lng"],  
            v["venue"]["location"]["distance"], 
            v["venue"]["categories"][0]["name"]) for v in results])
        count = count + 1
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ["Point Name",
                  "Neighborhood Latitude", 
                  "Neighborhood Longitude", 
                  "Venue", 
                  "Venue Latitude", 
                  "Venue Longitude", 
                  "Venue Distance",           
                  "Venue Category"]
    
    return(nearby_venues)

In [ ]:
sf_lat=37.7749
sf_lon=-122.4194
ch_lat=41.8781
ch_lon=-87.6298
la_lat=34.0522
la_lon=-118.2437
starbucks_m_lat=37.744644
starbucks_m_lon=-122.452731
import random

#37.780253, -122.509117
#37.804203, -122.459264 - upper left of starbucks intensive space
#37.767547, -122.409342 - lower right ofstarbucksintensive space
llat=37.767547
ulat=37.804203
llon=-122.459264
ulon=-122.409342
random_lat0 = random.uniform(37.767547,37.804203)
random_lon0 = random.uniform(-122.459264, -122.409342)
random_lat1 = random.uniform(llat,ulat)
random_lon1 = random.uniform(llon, ulon)
random_lat2 = random.uniform(llat,ulat)
random_lon2 = random.uniform(llon, ulon)
random_lat3 = random.uniform(llat,ulat)
random_lon3 = random.uniform(llon, ulon)
random_lat4 = random.uniform(llat,ulat)
random_lon4 = random.uniform(llon, ulon)
random_lat5 = random.uniform(llat,ulat)
random_lon5 = random.uniform(llon, ulon)
random_lat6 = random.uniform(llat,ulat)
random_lon6 = random.uniform(llon, ulon)
random_lat7 = random.uniform(llat,ulat)
random_lon7 = random.uniform(llon, ulon)
random_lat8 = random.uniform(llat,ulat)
random_lon8 = random.uniform(llon, ulon)
random_lat9 = random.uniform(llat,ulat)
random_lon9 = random.uniform(llon, ulon)

data = pd.DataFrame(np.array([[sf_lat, sf_lon], [ch_lat, ch_lon], [la_lat, la_lon], [starbucks_m_lat,starbucks_m_lon], [random_lat0,random_lon0], [random_lat1,random_lon1], [random_lat2,random_lon2], [random_lat3,random_lon3], [random_lat4,random_lon4], [random_lat5,random_lon5], [random_lat6,random_lon6], [random_lat7,random_lon7], [random_lat8,random_lon8], [random_lat9,random_lon9] ]), columns=['Latitude', 'Longitude'])

data

In [ ]:
venues = getNearbyVenues(latitudes = data["Latitude"],
                               longitudes = data["Longitude"]
                               )
venues

In [ ]:
venues['Venue'].unique()
venues['Venue'].value_counts()

In [ ]:
venues[350:400]


# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['point_name'] = venues['Point Name'] 
onehot['point_lat'] = venues['Neighborhood Latitude'] 
onehot['point_lon'] = venues['Neighborhood Longitude'] 
onehot['isStarbucks'] = (venues['Venue'] == "Starbucks")
# move neighborhood column to the first column
fixed_columns = [onehot.columns[-3]] + [onehot.columns[-2]] + [onehot.columns[-1]] + list(onehot.columns[:-3])
onehot = onehot[fixed_columns]

onehot.isStarbucks = onehot.isStarbucks.astype(int)

onehot.head()
onehot['isStarbucks'].value_counts()

In [ ]:
venues.head()

In [ ]:
grouped = onehot.groupby('point_name').mean().reset_index()
grouped

In [ ]:
num_top_venues = 50

for point in grouped['point_name']:
    print("----"+point+"----")
    temp = grouped[grouped['point_name'] == point].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
grouped.head()

In [ ]:
y = grouped['isStarbucks'] > 0
y = pd.DataFrame(y)
y = y.isStarbucks.astype(int)
print(y.value_counts())
X = grouped
print(X.shape)
X .drop(columns=['point_name', 'point_lat', 'point_lon', 'isStarbucks'], inplace=True)
print(X.columns)
print(X.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
import sklearn as sk  
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)  
RF.fit(X, y)  
RF.predict(X)  
RF.score(X,y)